Notes: need to re-run the whole thing for proper testing . change top -k to something smaller to reduce usage. Ensure to use the full testset too. Vary parameters a little as well.

you might need to regenerate the test set for each method that you use because the reference contexts will be different..

In [8]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.core.evaluation import (
    DatasetGenerator,
    FaithfulnessEvaluator,
    RelevancyEvaluator
)
from llama_index.llms.azure_openai import AzureOpenAI

import openai
import time

# Load environment variables from .env file
import pandas as pd
from llama_index.core.evaluation import (
    RetrieverEvaluator,
    get_retrieval_results_df,
)
import os
import logging
import pandas as pd
from dotenv import load_dotenv
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.node_parser import SentenceSplitter
from datetime import datetime
from llama_index.core import (
    StorageContext, VectorStoreIndex, SimpleDirectoryReader, 
    get_response_synthesizer, Settings
)
import traceback
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
from llama_index.core.evaluation import generate_question_context_pairs, QueryResponseDataset
from llama_index.llms.openai import OpenAI

import nest_asyncio

nest_asyncio.apply()

from pathlib import Path
from llama_index.readers.file import PDFReader
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
import json
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.core.embeddings import resolve_embed_model

######
import nest_asyncio
from llama_index.core import Settings

nest_asyncio.apply()
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
)

from giskard.rag import KnowledgeBase, generate_testset, QATestset
from giskard.rag import AgentAnswer
from giskard.rag import evaluate, RAGReport
from giskard.rag.question_generators import complex_questions, complex_situational_questions, compare_questions, simple_questions, rule_conclusion_questions, distracting_questions, na_questions, vague_questions, oos_questions, situational_questions, double_questions, conversational_questions
from giskard.rag.metrics.ragas_metrics import ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision


load_dotenv()
# Fetch API keys from environment variables
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-large-en-v1.5"
)

Settings.embed_model=embed_model

def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]


In [9]:
### TRY AGAIN
# setup LLM
import giskard
from llama_index.llms.azure_openai import AzureOpenAI
from giskard.llm import set_llm_model, set_llm_api
from giskard.llm.client import get_default_client
from giskard.llm.client import set_llm_api, set_llm_model
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
import os

# Set api keys here

os.environ["OPENAI_API_VERSION"] = "2024-02-01"
os.environ["AZURE_OPENAI_CHATGPT_DEPLOYMENT"] = "gpt4o"

set_llm_api("azure")
set_llm_model('gpt4o')

llm_gpt4o = AzureOpenAI(
    deployment_name="gpt4o",
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)

# Azure OpenAI Configuration
Settings.llm = llm_gpt4o

# You'll need to provide the name of the model that you've deployed
# Beware, the model provided must be capable of using function calls
client = get_default_client()
assert client._client._base_url == f'{os.environ["AZURE_OPENAI_ENDPOINT"]}/openai/'
assert client._client.api_key == os.environ["AZURE_OPENAI_API_KEY"]
assert client._client._api_version == os.environ["OPENAI_API_VERSION"]
assert client.model == "gpt4o"#os.environ["AZURE_OPENAI_CHATGPT_DEPLOYMENT"]


### Loading docs

In [10]:
import pandas as pd
import warnings
pd.set_option("display.max_colwidth", 400)
warnings.filterwarnings('ignore')

from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter, SemanticSplitterNodeParser
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.base.llms.types import ChatMessage, MessageRole
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings


loader = PyMuPDFReader()
#file_extractor = {".pdf": loader}
ipcc_documents1 = loader.load(file_path="../legal_data/LL144/LL144.pdf")
ipcc_documents2 = loader.load(file_path="../legal_data/LL144/LL144_Definitions.pdf")

documents = ipcc_documents1 + ipcc_documents2



## Semantic Splitter Method

In [11]:
chunk_size = 512
buffer_size = 11

base_splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=0)
splitter = SemanticSplitterNodeParser(embed_model=embed_model, buffer_size=buffer_size, 
                                     breakpoint_percentile_threshold=95)

nodes = splitter.get_nodes_from_documents(documents=documents)
index = VectorStoreIndex(nodes)

#text_nodes = splitter.build_semantic_nodes_from_documents(documents=documents)
#index = VectorStoreIndex.build_index_from_nodes(nodes=text_nodes)

#index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
chat_engine = index.as_chat_engine()

#text_nodes = splitter.get_nodes_from_documents(documents)#splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)

#testset = QATestset.load("../giskard_test_sets/LL144_300_balanced_first_5.jsonl")
retriever = index.as_retriever(similarity_top_k=10)

In [14]:
testset = generate_testset(knowledge_base,
                           question_generators=[simple_questions, distracting_questions],
                           num_questions=5,
                           language='en',
                           agent_description="A legal chatbot for Local Law 144 and other policy documents")


2024-07-17 19:16:56,357 pid:5642 MainThread giskard.rag  INFO     Finding topics in the knowledge base.


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


2024-07-17 19:17:02,451 pid:5642 MainThread giskard.rag  INFO     Found 3 topics in the knowledge base.


Generating questions: 100%|██████████| 5/5 [00:18<00:00,  3.76s/it]


In [15]:


def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] =="user" else MessageRole.ASSISTANT,
                                                          content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)


def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    # Format appropriately the history for your RAG agent
    messages = history if history else []
    messages.append({"role": "user", "content": question})

    # Get the answer and the documents
    #agent_output = get_answer_from_agent(messages)
    answer = answer_fn(question, history)
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]
    #print(documents)

    # Following llama_index syntax, you can get the answer and the retrieved documents
    #answer = agent_output.text
    #documents = agent_output.source_nodes

    # Instead of returning a simple string, we return the AgentAnswer object which
    # allows us to specify the retrieved context which is used by RAGAS metrics
    return AgentAnswer(
        message=answer,
        documents=documents
    )

report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])

Asking questions to the agent:   0%|          | 0/5 [00:00<?, ?it/s]

RagasMetric evaluation: 100%|██████████| 5/5 [02:43<00:00, 32.65s/it]


In [16]:
report

Loading BokehJS ...

In [17]:
report.to_html("ex.html")

## Semantic Double Merging Chunking

In [18]:
!pip install spacy -q
!python3 -m spacy download en_core_web_md -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 28.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [20]:
from llama_index.core.node_parser import (
    SemanticDoubleMergingSplitterNodeParser,
    LanguageConfig,
)
from llama_index.core import SimpleDirectoryReader

config = LanguageConfig(language="english", spacy_model="en_core_web_md")
splitter = SemanticDoubleMergingSplitterNodeParser(
    language_config=config,
    initial_threshold=0.4,
    appending_threshold=0.5,
    merging_threshold=0.5,
    max_chunk_size=5000,
)

nodes = splitter.get_nodes_from_documents(documents)
#print(nodes[0].get_content())
index = VectorStoreIndex(nodes)

#text_nodes = splitter.build_semantic_nodes_from_documents(documents=documents)
#index = VectorStoreIndex.build_index_from_nodes(nodes=text_nodes)

#index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
chat_engine = index.as_chat_engine()

#text_nodes = splitter.get_nodes_from_documents(documents)#splitter(documents)
knowledge_base_df = pd.DataFrame([node.text for node in nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)

#testset = QATestset.load("../giskard_test_sets/LL144_300_balanced_first_5.jsonl")
retriever = index.as_retriever(similarity_top_k=5)

report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])

KeyboardInterrupt: 

## Hierarchical Node Parser 

from llama_index.core.node_parser import HierarchicalNodeParser

node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)


In [28]:
# Load environment variables from .env file
import pandas as pd
from llama_index.core.evaluation import (
    RetrieverEvaluator,
    get_retrieval_results_df,
)
import os
import logging
import pandas as pd
from dotenv import load_dotenv
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.node_parser import SentenceSplitter
from datetime import datetime
from llama_index.core import (
    StorageContext, VectorStoreIndex, SimpleDirectoryReader, 
    get_response_synthesizer, Settings
)
import traceback
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
from llama_index.core.evaluation import generate_question_context_pairs, QueryResponseDataset
from llama_index.llms.openai import OpenAI

import nest_asyncio

nest_asyncio.apply()

from pathlib import Path
from llama_index.readers.file import PDFReader
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
import json
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.core.embeddings import resolve_embed_model

######
import nest_asyncio
from llama_index.core import Settings





nest_asyncio.apply()
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
)


load_dotenv()
# Fetch API keys from environment variables
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_API_VERSION = os.getenv("AZURE_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


set_llm_api("azure")
set_llm_model('gpt4o')

llm_gpt4o = AzureOpenAI(
    deployment_name="gpt4o",
    temperature=0, 
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_API_VERSION
)
llm = llm_gpt4o
# Azure OpenAI Configuration
Settings.llm = llm

# You'll need to provide the name of the model that you've deployed
# Beware, the model provided must be capable of using function calls
client = get_default_client()
assert client._client._base_url == f'{os.environ["AZURE_OPENAI_ENDPOINT"]}/openai/'
assert client._client.api_key == os.environ["AZURE_OPENAI_API_KEY"]
assert client._client._api_version == os.environ["OPENAI_API_VERSION"]
assert client.model == "gpt4o"#os.environ["AZURE_OPENAI_CHATGPT_DEPLOYMENT"]

embed_model = HuggingFaceEmbedding(
    #model_name = "Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True
    model_name="BAAI/bge-large-en-v1.5"
)
Settings.embed_model = embed_model

In [23]:
## TESTING METADATA METHOD: 

loader = PyMuPDFReader()
docs1 = loader.load(file_path="../legal_data/LL144/LL144.pdf")
docs2 = loader.load(file_path="../legal_data/LL144/LL144_Definitions.pdf")
docs = docs1 + docs2

#docs = loader.load(file_path="./EUAIACT.pdf")
#docs = docs[:20]
node_parser = SentenceSplitter(chunk_size=512)
base_nodes = node_parser.get_nodes_from_documents(docs)

extractors = [
    SummaryExtractor(summaries=["self"], show_progress=True),
    QuestionsAnsweredExtractor(questions=5, show_progress=True),
]

# run metadata extractor across base nodes, get back dictionaries
node_to_metadata = {}
for extractor in extractors:
    metadata_dicts = extractor.extract(base_nodes)
    for node, metadata in zip(base_nodes, metadata_dicts):
        if node.node_id not in node_to_metadata:
            node_to_metadata[node.node_id] = metadata
        else:
            node_to_metadata[node.node_id].update(metadata)

# cache metadata dicts
def save_metadata_dicts(path, data):
    with open(path, "w") as fp:
        json.dump(data, fp)


def load_metadata_dicts(path):
    with open(path, "r") as fp:
        data = json.load(fp)
    return data

save_metadata_dicts("metadata_dicts.json", node_to_metadata)
metadata_dicts = load_metadata_dicts("metadata_dicts.json")


# all nodes consists of source nodes, along with metadata
import copy

all_nodes = copy.deepcopy(base_nodes)
for node_id, metadata in node_to_metadata.items():
    for val in metadata.values():
        all_nodes.append(IndexNode(text=val, index_id=node_id))

all_nodes_dict = {n.node_id: n for n in all_nodes}

## Load index into vector index
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI


vector_index_metadata = VectorStoreIndex(all_nodes)



100%|██████████| 28/28 [00:17<00:00,  1.64it/s]


**Node ID:** ddfe51d4-6be9-49f6-93c4-8930357a69ce<br>**Similarity:** 0.7097790189792402<br>**Text:** 2 
 
 
Clarifying that the required “impact ratio” must be calculated separately to compare sex categories, 
race/ethnicity categories, and intersectional categories; 
 
Clarifying the types of data that may be used to conduct a bias audit; 
 
Clarifying that multiple employers using the same AEDT may rely on the same bias audit so long as they 
provide historical data, if available, for the independent auditor to consider in such bias audit; and 
 
Clarifying that an AEDT may not be used if its most recent bias audit is more than a year old; 
 
The Department received comments about the second version of the proposed rule from the public, including 
from employers, employment agencies, law firms, AEDT developers, and advocacy organizations. Various 
issues raised in the comments resulted in changes that are present in these final rules. These changes include: 
 
Modifying the definition of “machine learning, statistical modeling, data analytics, or artificial intelligence” 
to expand its scope; 
 
Adding a requirement that the bias audit indicate the number of individuals the AEDT assessed that are 
not included in the calculations because they fall within an unknown category, and requiring that number 
be included in the summary of results; 
 
Allowing an independent auditor to exclude a category that comprises less than 2% of the data being 
used for the bias audit from the calculations of impact ratio; 
 
Clarifying the examples of a bias audit; 
 
Clarifying when an employer or employment agency may rely on a bias audit conducted using the 
historical data of other employers or employment agencies; 
 
Providing examples of when an employer or employment agency may rely on a bias audit conducted with 
historical data, test data, or historical data from other employers and employment agencies; 
 
Clarifying that the number of applicants in a category and scoring rate of a category, if applicable, must 
be included in the summary of results. 
 
New m...<br>

**Node ID:** 733f554b-58a8-454e-9b03-4d586ac7f0fc<br>**Similarity:** 0.6986493140618864<br>**Text:** 5 
 
 
Test data. “Test data” means data used to conduct a bias audit that is not historical data. 
§ 5-301 Bias Audit. 
(a) An employer or employment agency may not use or continue to use an AEDT if more than one year has 
passed since the most recent bias audit of the AEDT.  
(b) Where an AEDT selects candidates for employment or employees being considered for promotion to 
move forward in the hiring process or classifies them into groups, a bias audit must, at a minimum: 
(1) Calculate the selection rate for each category; 
(2) Calculate the impact ratio for each category; 
(3) Ensure that the calculations required in paragraphs (1) and (2) of this subdivision separately 
calculate the impact of the AEDT on: 
i. Sex categories (e.g., impact ratio for selection of male candidates vs female candidates),  
ii. Race/Ethnicity categories (e.g., impact ratio for selection of Hispanic or Latino 
candidates vs Black or African American [Not Hispanic or Latino] candidates), and 
iii. intersectional categories of sex, ethnicity, and race (e.g., impact ratio for selection of 
Hispanic or Latino male candidates vs. Not Hispanic or Latino Black or African American 
female candidates). 
(4) Ensure that the calculations in paragraphs (1), (2), and (3) of this subdivision are performed for 
each group, if an AEDT classifies candidates for employment or employees being considered 
for promotion into specified groups (e.g., leadership styles); and 
(5) Indicate the number of individuals the AEDT assessed that are not included in the required 
calculations because they fall within an unknown category. 
 
Example: An employer wants to use an AEDT to screen resumes and schedule interviews for a job 
posting. To do so, the employer must ensure that a bias audit of the AEDT was conducted no more than 
a year before the planned use of the AEDT. This bias audit is necessary even though the employer is not 
using the AEDT to make the final hiring decision, but only to screen at an earl...<br>

**Node ID:** f94523f4-0f84-48ac-b3dc-b7f2b78b3ba1<br>**Similarity:** 0.6974879358135971<br>**Text:** These rules establish that a bias audit of an AEDT must calculate the selection rate for each race/ethnicity and 
sex category that is required to be reported on to the U.S. Equal Employment Opportunity Commission (“EEOC”) 
pursuant to the EEO Component 1 report, and compare the selection rates to the most selected category to 
determine an impact ratio. These calculations are consistent with Section 1607.4 of the EEOC Uniform 
Guidelines on Employee Selection Procedures. See 29 C.F.R. § 1607.4. These rules generally clarify obligations 
of employers and employment agencies under the new law. 
 
Specifically, these new rules will: 
 
Define terms; 
 
Clarify the requirements for a bias audit;  
 
Clarify the requirements for the published results of the required bias audit;  
 
Clarify the requirements for notices that employers and employment agencies must provide to employees 
and candidates for employment; and  
 
Clarify other obligations for the employer or employment agency.  
 
The Department initially proposed a version of these rules in September 2022. The Department received 
comments from the public, including from employers, employment agencies, law firms, AEDT developers, and 
advocacy organizations. Various issues raised in the comments led to a second version of the proposed rules, 
published in December 2022. Those changes included:  
 
Modifying the definition of AEDT to ensure greater focus; 
 
Clarifying that an “independent auditor” may not be employed or have a financial interest in an employer 
or employment agency that seeks to use or continue to use an AEDT or in a vendor that developed or 
distributed the AEDT; 
 
Revising the required calculation to be performed where an AEDT scores candidates;<br>

**Node ID:** 1fd9712d-290a-45a5-a07f-dc5c01a65212<br>**Similarity:** 0.6896891213402468<br>**Text:** (d) Notwithstanding the requirements of paragraphs (2) and (3) of subdivision (b) and paragraphs (3) and 
(4) of subdivision (c), an independent auditor may exclude a category that represents less than 2% of 
the data being used for the bias audit from the required calculations for impact ratio. Where such a 
category is excluded, the summary of results must include the independent auditor’s justification for the 
exclusion, as well as the number of applicants and scoring rate or selection rate for the excluded 
category.  
 
Example: An employer uses an AEDT to score applicants for “culture fit.” To do so, the employer must 
ensure that a bias audit of the AEDT was conducted no more than a year before the use of the AEDT. 
The employer provides historical data on “culture fit” score of applicants for each category to an 
independent auditor to conduct a bias audit as follows:  
 
Sex Categories 
  
# of 
Applicants 
Scoring 
Rate 
Impact Ratio 
Male 
92 
54.3% 
1.00 
Female 
76 
44.7% 
0.82 
 
 
Race/Ethnicity Categories 
  
# of 
Applicants 
Scoring 
Rate 
Impact 
Ratio 
Hispanic or Latino 
28 
64.2% 
1.00 
White (Not Hispanic or Latino) 
40 
37.5% 
0.58 
Black or African American (Not Hispanic or Latino) 
32 
50.0% 
0.78 
Native Hawaiian or Pacific Islander (Not Hispanic or Latino) 
8 
62.5% 
0.97 
Asian (Not Hispanic or Latino) 
24 
41.7% 
0.65 
Native American or Alaska Native (Not Hispanic or Latino) 
16 
62.5% 
0.97 
Two or More Races (Not Hispanic or Latino) 
20 
50.0% 
0.78<br>

In [34]:
vector_retriever_metadata = vector_index_metadata.as_retriever(
    similarity_top_k=3
)

retriever_metadata = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_metadata},
    node_dict=all_nodes_dict,
    verbose=False,
)

nodes = retriever_metadata.retrieve(
    "Can you tell me about a Bias Audit?"
)
for node in nodes:
    display_source_node(node, source_length=2000)


query_engine_metadata = RetrieverQueryEngine.from_args(
    retriever_metadata, llm=llm
)

**Node ID:** ddfe51d4-6be9-49f6-93c4-8930357a69ce<br>**Similarity:** 0.7097790189792402<br>**Text:** 2 
 
 
Clarifying that the required “impact ratio” must be calculated separately to compare sex categories, 
race/ethnicity categories, and intersectional categories; 
 
Clarifying the types of data that may be used to conduct a bias audit; 
 
Clarifying that multiple employers using the same AEDT may rely on the same bias audit so long as they 
provide historical data, if available, for the independent auditor to consider in such bias audit; and 
 
Clarifying that an AEDT may not be used if its most recent bias audit is more than a year old; 
 
The Department received comments about the second version of the proposed rule from the public, including 
from employers, employment agencies, law firms, AEDT developers, and advocacy organizations. Various 
issues raised in the comments resulted in changes that are present in these final rules. These changes include: 
 
Modifying the definition of “machine learning, statistical modeling, data analytics, or artificial intelligence” 
to expand its scope; 
 
Adding a requirement that the bias audit indicate the number of individuals the AEDT assessed that are 
not included in the calculations because they fall within an unknown category, and requiring that number 
be included in the summary of results; 
 
Allowing an independent auditor to exclude a category that comprises less than 2% of the data being 
used for the bias audit from the calculations of impact ratio; 
 
Clarifying the examples of a bias audit; 
 
Clarifying when an employer or employment agency may rely on a bias audit conducted using the 
historical data of other employers or employment agencies; 
 
Providing examples of when an employer or employment agency may rely on a bias audit conducted with 
historical data, test data, or historical data from other employers and employment agencies; 
 
Clarifying that the number of applicants in a category and scoring rate of a category, if applicable, must 
be included in the summary of results. 
 
New m...<br>

**Node ID:** 733f554b-58a8-454e-9b03-4d586ac7f0fc<br>**Similarity:** 0.6986493140618864<br>**Text:** 5 
 
 
Test data. “Test data” means data used to conduct a bias audit that is not historical data. 
§ 5-301 Bias Audit. 
(a) An employer or employment agency may not use or continue to use an AEDT if more than one year has 
passed since the most recent bias audit of the AEDT.  
(b) Where an AEDT selects candidates for employment or employees being considered for promotion to 
move forward in the hiring process or classifies them into groups, a bias audit must, at a minimum: 
(1) Calculate the selection rate for each category; 
(2) Calculate the impact ratio for each category; 
(3) Ensure that the calculations required in paragraphs (1) and (2) of this subdivision separately 
calculate the impact of the AEDT on: 
i. Sex categories (e.g., impact ratio for selection of male candidates vs female candidates),  
ii. Race/Ethnicity categories (e.g., impact ratio for selection of Hispanic or Latino 
candidates vs Black or African American [Not Hispanic or Latino] candidates), and 
iii. intersectional categories of sex, ethnicity, and race (e.g., impact ratio for selection of 
Hispanic or Latino male candidates vs. Not Hispanic or Latino Black or African American 
female candidates). 
(4) Ensure that the calculations in paragraphs (1), (2), and (3) of this subdivision are performed for 
each group, if an AEDT classifies candidates for employment or employees being considered 
for promotion into specified groups (e.g., leadership styles); and 
(5) Indicate the number of individuals the AEDT assessed that are not included in the required 
calculations because they fall within an unknown category. 
 
Example: An employer wants to use an AEDT to screen resumes and schedule interviews for a job 
posting. To do so, the employer must ensure that a bias audit of the AEDT was conducted no more than 
a year before the planned use of the AEDT. This bias audit is necessary even though the employer is not 
using the AEDT to make the final hiring decision, but only to screen at an earl...<br>

**Node ID:** f94523f4-0f84-48ac-b3dc-b7f2b78b3ba1<br>**Similarity:** 0.6974879358135971<br>**Text:** These rules establish that a bias audit of an AEDT must calculate the selection rate for each race/ethnicity and 
sex category that is required to be reported on to the U.S. Equal Employment Opportunity Commission (“EEOC”) 
pursuant to the EEO Component 1 report, and compare the selection rates to the most selected category to 
determine an impact ratio. These calculations are consistent with Section 1607.4 of the EEOC Uniform 
Guidelines on Employee Selection Procedures. See 29 C.F.R. § 1607.4. These rules generally clarify obligations 
of employers and employment agencies under the new law. 
 
Specifically, these new rules will: 
 
Define terms; 
 
Clarify the requirements for a bias audit;  
 
Clarify the requirements for the published results of the required bias audit;  
 
Clarify the requirements for notices that employers and employment agencies must provide to employees 
and candidates for employment; and  
 
Clarify other obligations for the employer or employment agency.  
 
The Department initially proposed a version of these rules in September 2022. The Department received 
comments from the public, including from employers, employment agencies, law firms, AEDT developers, and 
advocacy organizations. Various issues raised in the comments led to a second version of the proposed rules, 
published in December 2022. Those changes included:  
 
Modifying the definition of AEDT to ensure greater focus; 
 
Clarifying that an “independent auditor” may not be employed or have a financial interest in an employer 
or employment agency that seeks to use or continue to use an AEDT or in a vendor that developed or 
distributed the AEDT; 
 
Revising the required calculation to be performed where an AEDT scores candidates;<br>

In [35]:
from giskard.rag import KnowledgeBase, generate_testset, QATestset
from giskard.rag import AgentAnswer
from giskard.rag import evaluate, RAGReport
from giskard.rag.question_generators import complex_questions, complex_situational_questions, compare_questions, simple_questions, rule_conclusion_questions, distracting_questions, na_questions, vague_questions, oos_questions, situational_questions, double_questions, conversational_questions


In [30]:
chat_engine = vector_index_metadata.as_chat_engine()
knowledge_base_df = pd.DataFrame([node.text for node in all_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)
retriever = retriever_metadata

testset = QATestset.load("../giskard_test_sets/LL144_300_balanced_first_5.jsonl")

def answer_fn(question, history=None):
    if history:
        answer = chat_engine.chat(question, chat_history=[ChatMessage(role=MessageRole.USER if msg["role"] =="user" else MessageRole.ASSISTANT,
                                                          content=msg["content"]) for msg in history])
    else:
        answer = chat_engine.chat(question, chat_history=[])
    return str(answer)


def get_answer_fn(question: str, history=None) -> str:
    """A function representing your RAG agent."""
    # Format appropriately the history for your RAG agent
    messages = history if history else []
    messages.append({"role": "user", "content": question})

    # Get the answer and the documents
    #agent_output = get_answer_from_agent(messages)
    answer = answer_fn(question, history)
    retrieved_nodes = retriever.retrieve(question)
    documents = [node.node.text for node in retrieved_nodes]
    #print(documents)

    # Following llama_index syntax, you can get the answer and the retrieved documents
    #answer = agent_output.text
    #documents = agent_output.source_nodes

    # Instead of returning a simple string, we return the AgentAnswer object which
    # allows us to specify the retrieved context which is used by RAGAS metrics
    return AgentAnswer(
        message=answer,
        documents=documents
    )

report = evaluate(get_answer_fn,
                testset=testset,
                knowledge_base=knowledge_base,
                metrics=[ragas_context_recall, ragas_faithfulness, ragas_answer_relevancy, ragas_context_precision])

RagasMetric evaluation: 100%|██████████| 5/5 [01:39<00:00, 19.83s/it]


In [31]:
report.to_html('ex.html')

2024-07-18 13:40:10,149 pid:5642 MainThread giskard.rag  INFO     Finding topics in the knowledge base.
2024-07-18 13:40:21,199 pid:5642 MainThread giskard.rag  INFO     Found 6 topics in the knowledge base.


In [32]:
results = report.to_pandas()
results.head()
results.to_csv("results.csv")

In [33]:
results.head()

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason,RAGAS Context Recall,RAGAS Faithfulness,RAGAS Answer Relevancy,RAGAS Context Precision
id,,,,,,,,,,,,
fc76bbf8-8014-49c7-94c7-8196f0d629f8,Where can a proceeding to recover a civil penalty be returnable to?,A proceeding to recover a civil penalty authorized by this subchapter can be returnable to any tribunal established within the office of administrative trials and hearings or within any agency of the city designated to conduct such proceedings.,Document 26: d. A proceeding to recover any civil penalty authorized by this subchapter is returnable to any tribunal\nestablished within the office of administrative trials and hearings or within any agency of the city designated to\nconduct such proceedings.\n§ 20-873 Enforcement. The corporation counsel or such other persons designated by the corporation\ncounsel\non behalf\nof\nthe\ndepart...,[],"{'question_type': 'simple', 'seed_document_id': 26, 'topic': 'Automated Employment Decision Tools', 'distracting_context': None, 'situational_context': None, 'original_questions': None, 'fake_fact': None}",A proceeding to recover a civil penalty can be returnable to any tribunal established within the Office of Administrative Trials and Hearings or within any agency of the city designated to conduct such proceedings.,True,,1.0,1.0,0.994709,1.000000
21a627c7-41a7-4e35-9230-db785ea3da33,What information does the bias audit summary need to include?,"The bias audit summary needs to include the source and explanation of the data used to conduct the bias audit, the number of individuals the AEDT assessed that fall within an unknown category, and the number of applicants or candidates, the selection or scoring rates, as applicable, and the impact ratios for all categories.",Document 19: 9 \n \nExample 2: An employment agency has been using an AEDT for 6 months. The bias audit the \nemployment agency relied on before its first use of the AEDT was conducted 10 months ago using test \ndata. The employment agency will need an updated bias audit if it will continue to use the AEDT once 12 \nmonths have passed since the bias audit it first relied on was conducted. The ...,[],"{'question_type': 'simple', 'seed_document_id': 19, 'topic': 'Bias Audit of AEDT', 'distracting_context': None, 'situational_context': None, 'original_questions': None, 'fake_fact': None}","A bias audit summary should include the following information:\n\n1. **Results of the Bias Audit**:\n - Scoring rates for different sex and race/ethnicity categories.\n - Impact ratios for these categories.\n\n2. **Exclusions**:\n - If any category is excluded from the bias audit calculations, the summary must provide the criteria and rationale for such exclusions.",False,"The agent's answer is missing key elements such as the source and explanation of the data used, the number of individuals assessed that fall within an unknown category, and the number of applicants or candidates.",1.0,1.0,0.974511,0.583333
3b3eb0d0-b011-498b-8bf2-1f75bf2b36b3,Where can a proceeding to recover a civil penalty be returnable?,A proceeding to recover a civil penalty authorized by this subchapter can be returnable to any tribunal established within the office of administrative trials and hearings or within any agency of the city designated to conduct such proceedings.,"Document 3: d. A proceeding to recover any civil penalty authorized by this subchapter is returnable to any tribunal established within the office of administrative\ntrials and hearings or within any agency of the city designated to conduct such proceedings.\n(L.L. 2021/144, 12/11/2021, eff. 1/1/2023)\n§ 20-873 Enforcement.\nThe corporation counsel or such other persons designated by the cor...",[],"{'question_type': 'simple', 'seed_document_id': 3, 'topic': 'Automated Employment Decision Tools', 'distracting_context': None, 'situational_context': None, 'original_questions': None,